## Cleaning the Dataset before running analysis

In [14]:
# importing dependencies required
import pandas as pd
from pathlib import Path
import numpy as np

In [15]:
# reading csv data into a suitable dataframe
csvpath=Path("Data_Companies_Salaries1.csv")
data_df=pd.read_csv(csvpath)
data_df.head()

,Company Name,Rating,Average Salary,Role
0,Tata Consultancy Services,3.9,"7,80,000",Data Scientist
1,IBM,4.0,"12,00,000",Data Scientist
2,Accenture,4.0,"10,51,857",Data Scientist
3,Mu Sigma,3.5,"7,25,000",Data Scientist
4,Infosys,3.3,"7,70,714",Data Scientist


### Let's understand our Data Set and find out some unqiue identifiers

In [16]:
# Finding the number of unique companies in our data set
unique_companies=data_df["Company Name"].unique()
len(unique_companies)

5060

In [17]:
# Finding out the number of unique roles in our data set
unique_roles=data_df["Role"].unique()
len(unique_roles)

188

In [18]:
# It's preferable to limit the roles present to those who earn an annual wage. We will have to discard any row that yeilds monthly, hourly salaries as well as Contractor or Intern roles.
# source code - https://www.statology.org/pandas-drop-rows-that-contain-string/
discard=["Monthly","Hourly", "Intern","Contractor","internship"]

final_data=data_df[~data_df.Role.str.contains("|".join(discard))]
len(final_data["Role"].unique())

62

In [19]:
# In the Salary column we contain ranges instead of an average. To simplify this, we will assume the lower level of each range is our average salary
# Depending on if the salary has L, Cr or T - we should separate the letter from the numerical to make it easy for processing

final_data[["Average Salary Numeric", "Average Salary Letter"]] = final_data["Average Salary"].str.extract(r"(\d+)([A-Za-z]+)")

final_data=final_data[["Company Name","Rating","Role","Average Salary","Average Salary Numeric","Average Salary Letter"]]
final_data.tail()

/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/4275263199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data[["Average Salary Numeric", "Average Salary Letter"]] = final_data["Average Salary"].str.extract(r"(\d+)([A-Za-z]+)")
/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/4275263199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data[["Average Salary Numeric", "Average Salary Letter"]] = final_data["Average Salary"].str.extract(r"(\d+)([A-Za-z]+)")


,Company Name,Rating,Role,Average Salary,Average Salary Numeric,Average Salary Letter
8422,Innodatatics,4.3,"Data Scientist, Data Analyst, Machine Learning...",1L -1L,1,L
8424,AlmaBetter,4.2,"Data Scientist, Data Analyst, Machine Learning...",6L -6L,6,L
8426,Apollo Tyres,3.9,"Data Scientist, Data Analyst, Machine Learning...",6L -6L,6,L
8428,Magicflare,2.4,"Data Scientist, Data Analyst, Machine Learning...",10L -11L,10,L
8429,Fresherworld.com,4.0,"Data Scientist, Data Analyst, Machine Learning...",4L -4L,4,L


In [20]:
# Let's convert the salary ranges to NaN values
rows_not_na=final_data[~final_data["Average Salary Numeric"].isna()]
indexes=rows_not_na.index

final_data.loc[indexes, "Average Salary"] = np.nan

#retrospective check to assess if there are any blank values later in the code
final_data["Rating"].count()

5442

In [21]:
# Splitting the dataframe into two tables. Top Table contains Average Salary values that aren't NaN
top_data=final_data.loc[final_data["Average Salary Numeric"].isna()]

#index 8167 contain a salary range in an unclear format. I'll be dropping that row for clarity. Source https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
top_data=top_data.drop([8167])

# converting average salary to float and removing commas
top_data["Average Salary"]=top_data["Average Salary"].str.replace(',', '').astype("float64")
top_data = top_data[["Company Name","Rating","Role","Average Salary"]]
top_data.count()

Company Name      967
Rating            967
Role              967
Average Salary    967
dtype: int64

In [22]:
# Creating the bottom table, where Average Salary in the final dataframe is listed as NaN
lower_data=final_data.loc[final_data["Average Salary"].isna()]

# Converting numeric to float
lower_data["Average Salary Numeric"]=lower_data["Average Salary Numeric"].astype("float64")
lower_data["Average Salary Letter"].unique()


# translating values of Average Values by multiplying Average Salary numeric by it's alphabetical Co-efficient. (L=100000, Cr=10000000, T=1000)

for x in lower_data["Average Salary Letter"]:
    if x=="L":
        lower_data["Average Salary"]=lower_data["Average Salary Numeric"]*100000
    elif x=="Cr":
        lower_data["Average Salary"]=lower_data["Average Salary Numeric"]*10000000
    elif x=="T":
        lower_data["Average Salary"]=lower_data["Average Salary Numeric"]*1000

# Reducing the number of columns to match top datafram

lower_data=lower_data[["Company Name","Rating","Role","Average Salary"]]
lower_data.count()


/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/3672504690.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lower_data["Average Salary Numeric"]=lower_data["Average Salary Numeric"].astype("float64")
/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/3672504690.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lower_data["Average Salary"]=lower_data["Average Salary Numeric"]*100000
/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/3672504690.py:17: Setting

Company Name      4579
Rating            4474
Role              4579
Average Salary    4579
dtype: int64

In [23]:
# Count for lower table shows lesser rows in Ratings column which means there are empty or null values. I'll be replacing these with "0"
lower_data["Rating"].fillna(0, inplace=True)
lower_data.count()

Company Name      4579
Rating            4579
Role              4579
Average Salary    4579
dtype: int64

In [24]:
# Now that we have cleaned our top and lower dataframes to contain similar formatting for average salary, we can append these tables to form one.
# source - https://www.geeksforgeeks.org/python-pandas-dataframe-append/
main_df=top_data.append(lower_data,ignore_index=True)
main_df

/var/folders/33/k2j4j20s7fb5j52jx4p6qr0w0000gn/T/ipykernel_84604/1549427889.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  main_df=top_data.append(lower_data,ignore_index=True)


,Company Name,Rating,Role,Average Salary
0,Tata Consultancy Services,3.9,Data Scientist,780000.0
1,IBM,4.0,Data Scientist,1200000.0
2,Accenture,4.0,Data Scientist,1051857.0
3,Mu Sigma,3.5,Data Scientist,725000.0
4,Infosys,3.3,Data Scientist,770714.0
...,...,...,...,...
5541,Innodatatics,4.3,"Data Scientist, Data Analyst, Machine Learning...",100000.0
5542,AlmaBetter,4.2,"Data Scientist, Data Analyst, Machine Learning...",600000.0
5543,Apollo Tyres,3.9,"Data Scientist, Data Analyst, Machine Learning...",600000.0
5544,Magicflare,2.4,"Data Scientist, Data Analyst, Machine Learning...",1000000.0


In [25]:
# Now that we've cleaned the data, we'll output the dataframe into it's own path, so it's easier to work from
main_df.to_csv("../Data_Salaries_Final.csv",index=False)